## LAB 2 DATA SCIENCE

### 1. Import libraries

#### Basic Libraries

In [23]:
import pandas as pd               # data handling
import matplotlib.pyplot as plt   # plotting
! pip install --upgrade graphistry

  Using cached graphistry-0.41.1-py3-none-any.whl.metadata (24 kB)
  Using cached numpy-2.2.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached palettable-3.3.3-py2.py3-none-any.whl.metadata (3.3 kB)
  Using cached pandas-2.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (91 kB)
  Using cached pyarrow-21.0.0-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (3.3 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached squarify-0.4.4-py3-none-any.whl.metadata (600 bytes)
  Using cached typing_extensions-4.14.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached packaging-25.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl.metadata (8.4 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cach

In [24]:
import graphistry

graphistry.register(
    api=3,
    protocol="http",
    server="10.10.61.18",
    personal_key_id="3BCKNXVZQ6",
    personal_key_secret="15W9R97BAZTU8KP8"
)

### 2. Load dataset

#### File paths

In [25]:
http_log_path = "http_20250707_0000.log.gz"
conn_log_path = "conn_20250707_0000.log.gz"

#### Load HTTP log

In [26]:
http_df = pd.read_json(http_log_path, lines=True, compression="gzip")

#### Load Conn log

In [27]:
conn_df = pd.read_json(conn_log_path, lines=True, compression="gzip")

#### Add total_bytes for conn log 

to know total traffic per connection

In [28]:
conn_df["total_bytes"] = conn_df["orig_bytes"] + conn_df["resp_bytes"]

### 3. Basic Exploration

#### HTTP Log

In [29]:
print("=== HTTP Log ===")
print(http_df.head())      # first 5 rows
print("Shape:", http_df.shape)
print("Columns:", http_df.columns.tolist())
print("\n")

=== HTTP Log ===
                            ts                 uid        id.orig_h  \
0  2025-07-07T00:00:06.596972Z  CWACBm1G7zF8OV3c6l  192.168.110.109   
1  2025-07-07T00:00:25.847799Z   C1IvuQTriLbdG8HEl  192.168.110.109   
2  2025-07-07T00:00:25.906773Z   C1IvuQTriLbdG8HEl  192.168.110.109   
3  2025-07-07T00:00:31.755894Z  COsrn048sIOy1jZ69k  192.168.110.109   
4  2025-07-07T00:01:09.476267Z  Cf7UNy2VQEURGw0Zy4  192.168.110.106   

   id.orig_p       id.resp_h  id.resp_p  trans_depth method  \
0      49857   150.171.27.11         80            1    GET   
1      49943  118.215.103.18         80            1    GET   
2      49943  118.215.103.18         80            2    GET   
3      49790  118.215.103.18         80            3    GET   
4      64095  149.154.167.41         80            1   POST   

                 host                                                uri  ...  \
0  edge.microsoft.com  /browsernetworktime/time/1/current?cup2key=2:4...  ...   
1   ocsp.digice

#### Conn Log

In [30]:
print("=== Conn Log ===")
print(conn_df.head())
print("Shape:", conn_df.shape)
print("Columns:", conn_df.columns.tolist())

=== Conn Log ===
                            ts                 uid       id.orig_h  id.orig_p  \
0  2025-07-06T23:59:23.226648Z  C41JRf1hXOvgKN6doh  192.168.100.67      58292   
1  2025-07-06T23:59:54.571217Z  CApDtP2S12r94hXAg6  192.168.100.67      43778   
2  2025-07-06T23:59:54.891340Z   CcXdOce8ESk2GeGC3  192.168.100.67      43792   
3  2025-07-06T23:59:50.441054Z  CYQEdk2PfpcURObqY5  192.168.100.10      25200   
4  2025-07-06T23:59:55.431337Z  CbWZsv2H2pT0DTmung  192.168.100.26      53538   

       id.resp_h  id.resp_p proto   duration  orig_bytes  resp_bytes  ...  \
0   103.18.3.114        443   tcp  31.335102         0.0         0.0  ...   
1  203.80.16.244        443   tcp   0.092693         0.0    368450.0  ...   
2  203.80.16.244        443   tcp   0.129504         0.0    368450.0  ...   
3   204.14.183.2         53   udp        NaN         NaN         NaN  ...   
4  17.253.61.202         80   tcp   0.012170         0.0         0.0  ...   

        resp_l2_addr             

### 4. Graphistry Visualization

#### HTTP Logs

##### a. HTTP Requests 

In [31]:
g_http = graphistry.bind(
    source='id.orig_h',       # Client IP
    destination='host',       # HTTP host (domain)
    edge_title='response_body_len',
    edge_weight='response_body_len'
)

g_http.plot(http_df)


[PlotterBase.py:2147 -      _table_to_arrow() ]   Failed memoization speedup attempt due to Pandas internal hash function failing. Continuing without memoization speedups.This is fine, but for speedups around skipping re-uploads of previously seen tables, try identifying which columns have types that Pandas cannot hash, and convert them to hashable types like strings.



#### Conn Logs

##### a. Network Sessions

In [32]:
g_conn = graphistry.bind(
    source='id.orig_h',        # Client IP
    destination='id.resp_h',   # Server IP
    edge_title='total_bytes',   # Hover shows total bytes
    edge_weight='total_bytes'   # Edge thickness proportional to total bytes
)

g_conn.plot(conn_df)


[PlotterBase.py:2147 -      _table_to_arrow() ]   Failed memoization speedup attempt due to Pandas internal hash function failing. Continuing without memoization speedups.This is fine, but for speedups around skipping re-uploads of previously seen tables, try identifying which columns have types that Pandas cannot hash, and convert them to hashable types like strings.



#### Merge HTTP + Conn logs

##### Merge logs on 'uid'

In [33]:
merged = pd.merge(http_df, conn_df, on='uid', how='inner', suffixes=('_http', '_conn'))

##### a. Total bytes from conn

In [34]:
merged["total_bytes"] = merged["orig_bytes"] + merged["resp_bytes"]

g_combined = graphistry.bind(
    source='id.orig_h',   # Client IP
    destination='host',   # HTTP host
    edge_title='total_bytes',
    edge_weight='total_bytes'
)

g_combined.plot(merged)


[PlotterBase.py:2147 -      _table_to_arrow() ]   Failed memoization speedup attempt due to Pandas internal hash function failing. Continuing without memoization speedups.This is fine, but for speedups around skipping re-uploads of previously seen tables, try identifying which columns have types that Pandas cannot hash, and convert them to hashable types like strings.


[requests.py:14 -   log_requests_error() ]   HTTP 422 error - response content (JSON): {
  "data": {
    "bindings": {
      "destination": "host",
      "edge_title": "total_bytes",
      "edge_weight": "total_bytes",
      "source": "id.orig_h"
    },
    "dataset_id": "5985b3d3d2d14207bdbaf4b3b13a58db",
    "error_message": "Edge binding 'source' points to a column named 'id.orig_h' that does not exist, is there a typo in column name? Available columns: Index(['ts_http', 'uid', 'id.orig_h_http', 'id.orig_p_http', 'id.resp_h_http',\n       'id.resp_p_http', 'trans_depth', 'method', 'host', 'uri', 'version',\n       '

HTTPError: 422 Client Error: Unprocessable Entity for url: http://10.10.61.18/api/v2/upload/datasets/5985b3d3d2d14207bdbaf4b3b13a58db/edges/arrow